In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smpl-x
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/ smplx

In [ ]:
import cv2
import smplx
import numpy as np
import matplotlib.pyplot as plt
from torchvision.io import read_image

from pytorch3d.io import load_obj
from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.structures import Meshes, packed_to_list

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PerspectiveCameras,
    PointLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    BlendParams,
    TexturesVertex,
    TexturesUV
)

In [ ]:
### Setup
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')

In [ ]:
### Load SMPL-X model as Pytorch3d mesh

# if true, use SMPL object for HUMBI ; otherwise use SMPLX model
HUMBI_SMPL = True
GENERATED = False

if HUMBI_SMPL:
    if GENERATED:
        TEX_PATH = drive_path + 'generated_maps/generated_images/sample_run_256-generated-87/'
    else:
        TEX_PATH = drive_path + 'humbi_maps/humbi_body_texture/body_texture_medians/'
        subject_subset = ['median_subject_1.png', 'median_subject_2.png', 'median_subject_3.png', 'median_subject_4.png']

    OBJ_PATH = drive_path + 'smpl_bodies/text_uv_coor_smpl.obj'

    smplx_tex_img = torch.Tensor()

    for texture in os.listdir(TEX_PATH):
        if GENERATED:
            tex_img = read_image(TEX_PATH + texture)
            tex_img = torch.moveaxis(tex_img, 0, 2).unsqueeze_(0).float() * 1.0/255
            smplx_tex_img = torch.cat((smplx_tex_img, tex_img))
        else:
            if texture in subject_subset:
                tex_img = read_image(TEX_PATH + texture)
                tex_img = torch.moveaxis(tex_img, 0, 2).unsqueeze_(0).float() * 1.0/255
                smplx_tex_img = torch.cat((smplx_tex_img, tex_img))

    mesh = load_obj(OBJ_PATH, load_textures=False)

    verts_uvs = mesh[2].verts_uvs.unsqueeze_(0)
    verts_uvs = torch.cat([verts_uvs] * smplx_tex_img.shape[0])
    faces_uvs = mesh[1].textures_idx.unsqueeze_(0)
    faces_uvs = torch.cat([faces_uvs] * smplx_tex_img.shape[0] )

    smplx_mesh_verts = mesh[0].unsqueeze_(0)
    smplx_mesh_verts = torch.cat([smplx_mesh_verts] * smplx_tex_img.shape[0] )
    smplx_mesh_faces = mesh[1].verts_idx.unsqueeze_(0)
    smplx_mesh_faces = torch.cat([smplx_mesh_faces] * smplx_tex_img.shape[0])

    smplx_texture = TexturesUV(maps=smplx_tex_img, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

else:
    SMPLX_MODEL_PATH = drive_path + 'smpl_bodies'

    smplx_model = smplx.SMPLXLayer(SMPLX_MODEL_PATH, gender='neutral')

    smplx_mesh_verts = smplx_model.forward()['vertices']
    smplx_mesh_faces = torch.Tensor(smplx_model.faces.astype('int')).type(torch.int32).unsqueeze(0)
    smplx_verts_colors = torch.ones_like(smplx_mesh_verts)
    smplx_texture = TexturesVertex(verts_features=smplx_verts_colors)

smplx_mesh = Meshes(smplx_mesh_verts, smplx_mesh_faces, smplx_texture)

In [ ]:
### Plotting function

# images : tensor of images
# figsize : size of the plot (tuple)
# save : is the image being saved (boolean)
# path_name : path under which the image will be saved if save=True
# wspace and hspace : spacing between images

default_path = drive_path + 'generated_maps/generated_images/plot.png'
def plotting(images, figsize=(20,20), save=False, path_name=default_path, wspace=0, hspace=0.025):
    assert(len(images.shape) == 4), 'tensor must have 4 channels (#images, r, g, b) : consider using tensor.unsqueeze(0)'
    dimension = images.shape[0]

    x = np.ceil( np.sqrt(dimension) )
    y = np.ceil( dimension/x )

    if save:
      print('The following image will be saved')

    plt.figure(figsize=figsize)
    for i in range(dimension):
        plt.subplot(x,y,i+1)
        plt.imshow(images[i].cpu().numpy())
        plt.axis('off')

    plt.subplots_adjust(wspace=0, hspace=0.025)

    if save:
        plt.savefig(path_name, bbox_inches='tight')

    plt.show()
    print()

    return

In [ ]:
### Image inpainting function

# image : image tensor
# inpainting_method : inpainting method ('telea' or 'ns')

# Return inpainted image as a tensor

def inpaint_img(image, inpainting_method='telea'):
    assert(inpainting_method == 'telea' or inpainting_method == 'ns'), "inpainting_method must be one of 'telea' or 'ns' (Navier-Stokes)"

    image = (image * 255).numpy().astype(np.uint8)

    dim = image[:,:,0].shape
    inpaint_mask = np.zeros(dim, dtype=np.uint8)
    mask_condition = (image[:,:,0] == 0) & (image[:,:,1] == 0) & (image[:,:,2] == 0)
    inpaint_mask[mask_condition] = 1

    # inpainting
    method = cv2.INPAINT_TELEA * (inpainting_method=='telea') + cv2.INPAINT_NS * (inpainting_method=='ns')
    inpainted_image = cv2.inpaint(image, inpaint_mask, 3, method)

    return torch.Tensor(inpainted_image.copy()).float() * 1.0/255

In [ ]:
### TexturesUV inpainting function

# textures : TexturesUV object
# inpainting_method : inpainting method ('telea' or 'ns')

# Return TexturesUV object with inpainted textures

def inpaint_texture(textures: TexturesUV, inpainting_method='telea'):
    assert(len(textures.maps_padded().shape) == 4), 'tensor must have 4 channels (#images, r, g, b) : consider using tensor.unsqueeze(0)'
    dimension = textures.maps_padded().shape[0]

    inpainted_textures = textures.clone()
    for i in range(dimension):
        inpainted_textures._maps_padded[i] = inpaint_img(inpainted_textures.maps_padded()[i], inpainting_method)

    return inpainted_textures

In [ ]:
### If HUMBI_SMPL = True, visualize the loaded texture

if HUMBI_SMPL:
    plotting( smplx_texture.maps_padded() )

    inpainted_smplx_texture = inpaint_texture(smplx_texture)
    plotting( inpainted_smplx_texture.maps_padded() )

    # plot mapping from vertices to texture for first subject
    plt.figure(figsize=(10, 10))
    texturesuv_image_matplotlib(smplx_texture[0], subsample=None)
    plt.axis("off")

In [ ]:
### Setup pytorch3d mesh renderer

def get_mesh_renderer(img_width=1024, img_height=1024, device=device):

    raster_settings = RasterizationSettings(
        image_size=(img_width, img_height), 
        blur_radius=0.0, 
        faces_per_pixel=1,
        max_faces_per_bin=10000
    )

    blend_params = BlendParams(background_color=(0, 0, 0))
    lights = PointLights(device=device, location=[[2.0, 2.0, 2.0]])

    cameras = PerspectiveCameras().to(device)

    mesh_renderer = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras, 
            raster_settings=raster_settings
        ),
        shader=SoftPhongShader(
            device=device, 
            cameras=cameras,
            lights=lights,
            blend_params=blend_params
        )
    )

    return mesh_renderer

In [ ]:
### Render test mesh

img_width, img_height = 1024, 1024
mesh_renderer = get_mesh_renderer(img_width, img_height, device)

smplx_mesh = smplx_mesh.to(device)

materials = Materials(device=device, specular_color=[[0.0, 0.0, 0.0]], shininess=0.0)

dist = 1 # 20
azim = 20 # 10
elev = 20 # 0
# up = ((0, 0, 1), )

# R, T = look_at_view_transform(dist=dist, elev=elev, azim=azim, up=up)
R, T = look_at_view_transform(dist=dist, elev=elev, azim=azim)

if HUMBI_SMPL:
    min_y = -0.2 ; max_y = 1.8
else:
    min_y = -1.4 ; max_y = 0.6

# cameras = FoVOrthographicCameras(R=R, T=T).to(device)
cameras = FoVOrthographicCameras(min_y=min_y, max_y=max_y, R=R, T=T).to(device)

img = mesh_renderer(smplx_mesh, cameras=cameras, materials=materials)[:, :, :, 0:3].detach()

plotting(img, figsize=(20,20))

if HUMBI_SMPL:
    final_render = drive_path + 'generated_maps/generated_images/sample_run_256.png'
    plotting(img[0].unsqueeze(0), figsize=(10,10), save=True, path_name=final_render)

In [ ]:
### Plotly (interactive) visualization

# Convert TexturesUV to TexturesVertex
def convert_to_textureVertex(textures_uv: TexturesUV, meshes:Meshes, idx: int) -> TexturesVertex:
    assert( idx in range( len(meshes.num_verts_per_mesh()) ) ), 'idx needs be an integer between 0 and #subjects'

    verts_colors_packed = torch.zeros_like(meshes.verts_packed()).to(device)
    verts_colors_packed[meshes.faces_packed()] = textures_uv.faces_verts_textures_packed().to(device)

    vertex_shape = meshes.num_verts_per_mesh()[idx].item()

    return TexturesVertex( packed_to_list(
        verts_colors_packed[idx * vertex_shape : (idx+1) * vertex_shape, :],
        meshes.num_verts_per_mesh()[idx] ) )

index = 1 # subject to visualize
vertex_texture = convert_to_textureVertex(smplx_texture, smplx_mesh, index)

mesh = Meshes(
    verts=smplx_mesh_verts[index, :, :].unsqueeze(0).to(device),   
    faces=smplx_mesh_faces[index, :, :].unsqueeze(0).to(device),
    textures=vertex_texture
)

# Render figure
fig = plot_scene({
    "Humbi generated texture mapped on SMPL body": {
        "plot": mesh
        }
},)

fig.show()